In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Experiment 1: Train Logistic Regression Classifier

In [16]:
params = {
    "solver":"lbfgs",
    "max_iter": 100,
    "multi_class":"auto",
    "random_state":8888
}
lr = LogisticRegression(**params)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [18]:
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_dict

{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.9560439560439561,
  'support': 270.0},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30.0},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300.0},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.9159951159951161,
  'support': 300.0}}

In [19]:
import mlflow

In [29]:
report_dict['macro avg']

{'precision': 0.7853260869565217,
 'recall': 0.7333333333333334,
 'f1-score': 0.7557997557997558,
 'support': 300.0}

In [ ]:
mlflow.set_experiment('First Experiment')
mlflow.set_tracking_uri('http://127.0.0.1:5000/')

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'precision_0': report_dict['0']['precision'],
        'precision_1': report_dict['1']['precision'],
        'macro_avg': report_dict['macro avg']['f1-score']
    })
    
    mlflow.sklearn.log_model(lr,"Logistic Regression")
    

2025/11/28 18:51:53 INFO mlflow.tracking.fluent: Experiment with name 'First Experiment' does not exist. Creating a new experiment.
2025/11/28 18:51:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/28 18:52:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run thoughtful-rook-541 at: http://127.0.0.1:5000/#/experiments/893726118916031007/runs/2317ac30b72746f78ca0a5d709942566
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/893726118916031007


In [14]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [6]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.63      0.76        30

    accuracy                           0.96       300
   macro avg       0.96      0.81      0.87       300
weighted avg       0.96      0.96      0.96       300



### Experiment 3: Train XGBoost

In [8]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [11]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



<h2 align="center" style="color:blue">Track Experiments Using MLFlow</h2>

In [33]:
models = [
    (
        "Logistic Regression",
        {"C": 1, "solver": "liblinear"},
        LogisticRegression(C=1, solver="liblinear"),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "Random Forest",
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(n_estimators=30, max_depth=3),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": "logloss"},
        XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
        (X_train, y_train),
        (X_test, y_test),
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": "logloss"},
        XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
        (X_train_res, y_train_res),
        (X_test, y_test),
    ),
]

In [ ]:
reports = []
fitted_models = [] 

for model_name, params, model, train_set, test_set in models:
    X_tr, y_tr = train_set
    X_te, y_te = test_set

    model.set_params(**params)
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)
    report = classification_report(y_te, y_pred, output_dict=True)
    reports.append(report)
    fitted_models.append((model_name, params, model, (X_tr, y_tr), (X_te, y_te)))

In [35]:
reports

[{'0': {'precision': 0.9834710743801653,
   'recall': 0.8814814814814815,
   'f1-score': 0.9296875,
   'support': 270.0},
  '1': {'precision': 0.4482758620689655,
   'recall': 0.8666666666666667,
   'f1-score': 0.5909090909090909,
   'support': 30.0},
  'accuracy': 0.88,
  'macro avg': {'precision': 0.7158734682245654,
   'recall': 0.8740740740740741,
   'f1-score': 0.7602982954545454,
   'support': 300.0},
  'weighted avg': {'precision': 0.9299515531490453,
   'recall': 0.88,
   'f1-score': 0.8958096590909091,
   'support': 300.0}},
 {'0': {'precision': 0.9840637450199203,
   'recall': 0.9148148148148149,
   'f1-score': 0.9481765834932822,
   'support': 270.0},
  '1': {'precision': 0.5306122448979592,
   'recall': 0.8666666666666667,
   'f1-score': 0.6582278481012658,
   'support': 30.0},
  'accuracy': 0.91,
  'macro avg': {'precision': 0.7573379949589398,
   'recall': 0.8907407407407408,
   'f1-score': 0.803202215797274,
   'support': 300.0},
  'weighted avg': {'precision': 0.9387185

In [ ]:
import mlflow
mlflow.set_experiment("Anomaly_Detection")
mlflow.set_tracking_uri("http://localhost:5000")

for i, (model_name, params, model, train_set, test_set) in enumerate(fitted_models):
    report = reports[i]
    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metric("accuracy", report.get("accuracy", 0.0))
        mlflow.log_metric("recall_class_1", report.get("1", {}).get("recall", 0.0))
        mlflow.log_metric("recall_class_0", report.get("0", {}).get("recall", 0.0))
        mlflow.log_metric("f1_score_macro", report.get("macro avg", {}).get("f1-score", 0.0))

        try:
            mlflow.sklearn.log_model(model, "model")
        except Exception as e:
            if isinstance(model, xgb.Booster):
                mlflow.xgboost.log_model(model, "model")
            else:
                raise
            mlflow.sklearn.log_model(model, "model")  

2025/12/01 11:08:39 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly_Detection' does not exist. Creating a new experiment.
2025/12/01 11:08:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/01 11:08:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/12/01 11:08:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/262343112074910306/runs/fd11eef0ae8a43e8888fb740f758d31e
🧪 View experiment at: http://localhost:5000/#/experiments/262343112074910306


2025/12/01 11:08:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/262343112074910306/runs/88779c6ef8264f82b24c67cd01fd31fd
🧪 View experiment at: http://localhost:5000/#/experiments/262343112074910306


2025/12/01 11:08:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/01 11:08:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/12/01 11:08:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/262343112074910306/runs/974f619a92b2401db7ed45175df7712b
🧪 View experiment at: http://localhost:5000/#/experiments/262343112074910306


2025/12/01 11:09:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/262343112074910306/runs/e4610b4f46294cf3b30ac4d26bd1372d
🧪 View experiment at: http://localhost:5000/#/experiments/262343112074910306


### Model Registry

In [ ]:
model_name = "Logistic_Regression"
run_id = input("Enter Run ID")
model_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(model_uri=model_uri, name=model_name)
print(result)

Successfully registered model 'Logistic_Regression'.
2025/12/01 11:32:42 WARNING mlflow.tracking._model_registry.fluent: Run with id fd11eef0ae8a43e8888fb740f758d31e has no artifacts at artifact path 'model', registering model based on models:/m-b6d79309e3a64131b0e2fef3c04fc071 instead
2025/12/01 11:32:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Logistic_Regression, version 1


<ModelVersion: aliases=[], creation_timestamp=1764570762087, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1764570762087, metrics=None, model_id=None, name='Logistic_Regression', params=None, run_id='fd11eef0ae8a43e8888fb740f758d31e', run_link='', source='models:/m-b6d79309e3a64131b0e2fef3c04fc071', status='READY', status_message=None, tags={}, user_id='', version='1'>


Created version '1' of model 'Logistic_Regression'.


## Load Challanger Model

In [49]:
model_version = 1
model_uri = f"models:/{model_name}@challenger"
loaded_model = mlflow.sklearn.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 1, 0])

### Production Environment

In [50]:
dev_model_uri = f"models:/{model_name}@challenger"
prod_model= "Anomaly Detection Production Model"
client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri,dst_name=prod_model)

Successfully registered model 'Anomaly Detection Production Model'.
Copied version '1' of model 'Logistic_Regression' to version '1' of model 'Anomaly Detection Production Model'.


<ModelVersion: aliases=[], creation_timestamp=1764571863695, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1764571863695, metrics=None, model_id=None, name='Anomaly Detection Production Model', params=None, run_id='fd11eef0ae8a43e8888fb740f758d31e', run_link='', source='models:/Logistic_Regression/1', status='READY', status_message=None, tags={}, user_id='', version='1'>